# **Launch Sites Locations Analysis with Folium**


The success rate of a rocket launch can be influenced by multiple factors, such as the payload mass, the type of orbit, and other mission-specific variables. Additionally, the geographical location of the launch site and its proximity to key features may also play an important role, as they affect the initial trajectory of the rocket. Identifying an optimal site for constructing a launch facility involves considering many such factors, and by analyzing the characteristics of existing launch sites, we may be able to uncover valuable insights into what contributes to higher launch success rates.

In the earlier exploratory data analysis notebooks, we used `Matplotlib` and `Seaborn` to visualize the SpaceX launch dataset. These visualizations helped us identify some preliminary patterns and correlations, particularly between launch sites and mission success rates. In this notebook, we will take the analysis a step further by performing interactive geospatial visualizations with `Folium`, allowing us to explore launch site locations and outcomes in a more dynamic and intuitive way.

## Goals


This notebook contains the following tasks:
- **STEP 1:** Mark all launch sites on a map
- **STEP 2:** Mark the success/failed launches for each site on the map
- **STEP 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this notebook:


In [23]:
# Install required packages if not already installed
!pip install folium pandas



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import folium as folium_lib
import pandas as pd_var

In [25]:
# Correct way: import the real plugin but alias it to your custom name
from folium.plugins import MarkerCluster as launch_marker
from folium.plugins import MousePosition as MousePosition_var
from folium.features import DivIcon as DivIcon_var


If you need to refresh your memory about folium, you may download and refer to this previous folium notebook:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/notebooks/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [26]:
# Download and read the `spacex_launch_geo.csv` file
import requests
import pandas as pd_var
import io

URL_var = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"

response_var = requests.get(URL_var)
spacex_df_var = pd_var.read_csv(io.StringIO(response_var.text))

# Preview data
spacex_df_var.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [27]:
# Select relevant columns: `Launch Site`, `Lat`, `Long`, `class`
spacex_df_var = spacex_df_var[['Launch Site', 'Lat', 'Long', 'class']]

# Group by launch site and take the first row for each site
launch_site_info = spacex_df_var.groupby(['Launch Site'], as_index=False).first()

# Keep only site name, latitude, and longitude
launch_site_info = launch_site_info[['Launch Site', 'Lat', 'Long']]

launch_site_info


,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [28]:
import folium

# Start location is NASA Johnson Space Center
nasa_coordinate_var = [29.559684888503615, -95.0830971930759]
launch_map_view = folium.Map(location=nasa_coordinate_var, zoom_start=10)


We could use `folium.Circle` to add a highlighted circle area with a text notebookel on a specific coordinate. For example, 


In [29]:
# Create a circle around NASA Johnson Space Center with a popup label
circle_var = folium_lib.Circle(
    location=nasa_coordinate_var,
    radius=1000,
    color='#D35400',
    fill=True
).add_child(folium_lib.Popup('NASA Johnson Space Center'))

# Create a marker at NASA Johnson Space Center with a text label
launch_marker = folium_lib.Marker(
    location=nasa_coordinate_var,
    icon=DivIcon_var(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#D35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)

# Add to the map
launch_map_view.add_child(circle_var)
launch_map_view.add_child(launch_marker)

launch_map_view


and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'notebookel', ))`


In [30]:
# Initialize the map centered around NASA JSC
launch_map_view = folium_lib.Map(location=nasa_coordinate_var, zoom_start=5)

# Loop through each launch site and add a circle + marker
for site_name, site_lat, site_lon in zip(launch_site_info['Launch Site'],
                                         launch_site_info['Lat'],
                                         launch_site_info['Long']):
    site_coordinates = [site_lat, site_lon]
    
    # Add a circle for each site
    circle_var = folium_lib.Circle(
        location=site_coordinates,
        radius=1000,
        color='#D35400',
        fill=True
    ).add_child(folium_lib.Popup(site_name))
    
    # Add a marker with text label
    launch_marker = folium_lib.Marker(
        location=site_coordinates,
        icon=DivIcon_var(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#D35400;"><b>%s</b></div>' % site_name,
        )
    )
    
    # Attach to map
    launch_map_view.add_child(circle_var)
    launch_map_view.add_child(launch_marker)

launch_map_view


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


## Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [31]:
spacex_df_var.tail(10)


,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [32]:
launch_marker = folium_lib.Marker(
    location=[site_lat, site_lon],
    icon=DivIcon_var(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#D35400;"><b>%s</b></div>' % site_name,
    )
)


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [33]:
# Add a column to assign marker colors based on launch outcome
# If class = 1 (successful launch), color is green
# If class = 0 (failed launch), color is red

def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df_var['marker_color'] = spacex_df_var['class'].apply(assign_marker_color)

# Show last 10 rows
spacex_df_var.tail(10)


,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [34]:
# Add markers for each launch in the dataframe
# Each marker will show the outcome (green = success, red = failure)

for idx, row in spacex_df_var.iterrows():
    coordinates = [row['Lat'], row['Long']]
    
    marker = folium_lib.Marker(
        location=coordinates,
        icon=folium_lib.Icon(color="white", icon_color=row['marker_color'])
    )
    
    marker.add_to(launch_map_view)

launch_map_view


From the color-notebookeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


## Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [35]:
from folium.plugins import MousePosition

# Formatter function in JavaScript (as a string)
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# Add mouse position plugin to show lat/lon on hover
mouse_position = MousePosition(
    position='topright',
    separator=' | Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter
)

launch_map_view.add_child(mouse_position)

launch_map_view


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [36]:
from math import sin, cos, sqrt, atan2, radians

def compute_distance(lat1, lon1, lat2, lon2):
    """
    Compute the great-circle distance between two points 
    on the Earth (specified in decimal degrees).
    """
    # Approximate radius of earth in km
    R = 6373.0

    # Convert coordinates from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Differences
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [37]:
# Launch site coordinates
launch_site_lat = 28.563197
launch_site_lon = -80.576820
#coast coord
coastline_lat = 28.56334
coastline_lon = -80.56799
distance_coastline = compute_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(distance_coastline,' km')


0.8627671182499878  km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [38]:
from folium.features import DivIcon

# Example with distance annotation
distance_km = compute_distance(site_lat, site_lon, coastline_lat, coastline_lon)

launch_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#D35400;"><b>{:.2f} km</b></div>'.format(distance_km),
    )
)
launch_map_view.add_child(launch_marker)
launch_map_view



_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [39]:
# Draw a straight line (polyline) between the launch site and the nearest coastline

# Define the coordinates
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]

# Create the line
line = folium.PolyLine(locations=coordinates, weight=1)

launch_map_view.add_child(launch_marker)
launch_map_view


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


In [40]:
import folium
from folium.features import DivIcon

# Define NASA JSC as the map center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=6)

# Define launch site
launch_coordinate = [28.562302, -80.57682]

# Add a marker for the launch site
folium.Marker(
    launch_coordinate,
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
).add_to(site_map)

site_map


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

In [41]:
# Define coordinates (lat, lon) for reference points
launch_coordinate = (28.5618571, -80.577366)  # Example: Kennedy Space Center
city = (28.538336, -81.379234)                # Example: Orlando, FL
railway = (28.5721, -80.5853)                 # Example coordinates near launch site
highway = (28.5623, -80.5708)                 # Example coordinates for highway
coastline_lat, coastline_lon = (28.4, -80.6)  # Example coastline point

# Function to compute distance (Haversine formula or your own)
from math import radians, sin, cos, sqrt, atan2

def compute_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# Compute distances
city_distance = compute_distance(launch_coordinate[0], launch_coordinate[1], city[0], city[1])
railway_distance = compute_distance(launch_coordinate[0], launch_coordinate[1], railway[0], railway[1])
highway_distance = compute_distance(launch_coordinate[0], launch_coordinate[1], highway[0], highway[1])
coastline_distance = compute_distance(launch_coordinate[0], launch_coordinate[1], coastline_lat, coastline_lon)

# Print results
print("City Distance:", round(city_distance, 2), "km")
print("Railway Distance:", round(railway_distance, 2), "km")
print("Highway Distance:", round(highway_distance, 2), "km")
print("Coastline Distance:", round(coastline_distance, 2), "km")


City Distance: 78.36 km
Railway Distance: 1.38 km
Highway Distance: 0.64 km
Coastline Distance: 18.13 km
